# FrameAxis analysis for r/self and r/nosleep

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  1


In [2]:
import pickle
with open('./Download/data_cleaned.pickle', 'rb') as handle:
    data_cleaned = pickle.load(handle)

# 1. Building a Set of Microframes

In [3]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/anthony/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/anthony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Show all antonyms in WordNet:

In [4]:
from nltk.corpus import wordnet as wn

def antonyms_for(word):
    antonyms = set()
    for ss in wn.synsets(word):
        for lemma in ss.lemmas():
            any_pos_antonyms = [ antonym.name() for antonym in lemma.antonyms() ]
            for antonym in any_pos_antonyms:
                antonym_synsets = wn.synsets(antonym)
                if wn.ADJ not in [ ss.pos() for ss in antonym_synsets ]:
                    continue
                antonyms.add(antonym)
    return antonyms

In [5]:
from nltk.corpus import wordnet as wn

wn_all_antonyms = set()

for i in wn.all_synsets():
    if i.pos() in ['a', 's']: # If synset is adj or satelite-adj.
        for j in i.lemmas(): # Iterating through lemmas for each synset.
            if j.antonyms(): # If adj has antonym.
                # Prints the adj-antonym pair.
                wn_all_antonyms.add((j.name(), j.antonyms()[0].name()))

In [6]:
len(wn_all_antonyms)

3531

### Import pretrained embeddings

### **Options**

`CLASStorchtext.vocab.GloVe(name='840B', dim=300, **kwargs)`

`CLASStorchtext.vocab.FastText(language='en', **kwargs)`

`CLASStorchtext.vocab.CharNGram(**kwargs)`

In [7]:
import torch
import torchtext
import numpy as np

In [8]:
glove = torchtext.vocab.GloVe(name="840B",dim=300)

In [9]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [10]:
from nltk.corpus import wordnet as wn

wn_all_antonyms_words = set()

for ant_pair in wn_all_antonyms:
    wn_all_antonyms_words = set(tuple(wn_all_antonyms_words) + ant_pair)

In [11]:
no_emb_words = set()
for adj in wn_all_antonyms_words:
    if torch.all(glove[adj] == torch.zeros(300)):
        no_emb_words.add(adj)

In [12]:
len(wn_all_antonyms)

3531

In [13]:
copy = wn_all_antonyms.copy()
for pair in wn_all_antonyms:
    pair1, pair2 = pair
    if pair1 in no_emb_words or pair2 in no_emb_words:
        copy.remove(pair)
        
wn_all_antonyms = copy

In [14]:
len(wn_all_antonyms)

3131

In [15]:
3531-3131

400

In [16]:
wn_all_antonyms

{('unpopular', 'popular'),
 ('lively', 'dull'),
 ('apocrine', 'eccrine'),
 ('fixed', 'unfixed'),
 ('inexpressible', 'expressible'),
 ('airworthy', 'unairworthy'),
 ('unmalted', 'malted'),
 ('communicative', 'uncommunicative'),
 ('uninhibited', 'inhibited'),
 ('punitive', 'rehabilitative'),
 ('paid', 'unpaid'),
 ('leaky', 'tight'),
 ('prudent', 'imprudent'),
 ('untruthful', 'truthful'),
 ('precise', 'imprecise'),
 ('wieldy', 'unwieldy'),
 ('unlisted', 'listed'),
 ('aperiodic', 'periodic'),
 ('nearsighted', 'farsighted'),
 ('crosswise', 'lengthwise'),
 ('unoiled', 'oiled'),
 ('thin', 'thick'),
 ('modulated', 'unmodulated'),
 ('irremediable', 'remediable'),
 ('divisible', 'indivisible'),
 ('uncrossed', 'crossed'),
 ('awake', 'asleep'),
 ('artless', 'artful'),
 ('sorrowful', 'joyful'),
 ('unclean', 'clean'),
 ('synchronous', 'asynchronous'),
 ('thermal', 'nonthermal'),
 ('agitated', 'unagitated'),
 ('anestrous', 'estrous'),
 ('efferent', 'afferent'),
 ('lengthwise', 'crosswise'),
 ('unfree

### Add Custom Antonym Pairs

In [17]:
from nltk.corpus import wordnet as wn

# add words here:
WORDS = ['man', 'human']

for word in WORDS:
    for syn in wn.synsets(word):
        for j in syn.lemmas(): # Iterating through lemmas for each synset.
            if j.antonyms():
                pair1 = j.name()
                pair2 = j.antonyms()[0].name()
                # don't add pair to the list if at least one of them has no glove embeddings
                if torch.all(glove[pair1] == torch.zeros(300)) or torch.all(glove[pair2] == torch.zeros(300)):
                    print(f"Pair {(j.name(), j.antonyms()[0].name())} cannot be added because one of them does not have GloVe embeddings.")
                else:
                    print(f"Added {(pair1, pair2)}")
                    wn_all_antonyms.add((pair1, pair2))

Added ('man', 'woman')
Added ('serviceman', 'civilian')
Added ('man', 'woman')
Added ('human', 'nonhuman')


In [18]:
microframes = wn_all_antonyms

In [19]:
len(microframes)

3133

### Semantic Axis Vector

In [20]:
semantic_axis_vectors = dict()
for mframe in microframes:
    semantic_axis_vectors['(' + ', '.join(mframe) + ')'] = np.expand_dims(glove[mframe[1]] - glove[mframe[0]], axis= 0)

In [21]:
semantic_axis_vectors['(man, woman)']

array([[ 1.98666990e-01,  7.22199827e-02, -1.86462998e-01,
         5.83739996e-01, -7.46250004e-02, -9.17997956e-03,
         3.19599956e-02,  3.92699987e-02,  1.38819993e-01,
         6.78999424e-02, -2.03620002e-01,  2.38368988e-01,
        -2.72006691e-01, -3.08981687e-01, -1.61559999e-01,
        -1.46412000e-01, -4.35483992e-01,  1.89160019e-01,
        -2.95740008e-01, -3.50700021e-02,  9.05001163e-03,
         2.55119979e-01, -1.56845003e-01, -3.59719992e-01,
        -2.88099945e-02,  4.00590003e-01,  1.07859999e-01,
        -2.12710008e-01, -3.14889997e-01, -1.76756993e-01,
         1.34200007e-01, -5.70900142e-02,  2.26850003e-01,
        -2.34786004e-01,  4.04179990e-01,  2.45397985e-01,
         1.38130009e-01, -3.17710042e-01, -6.44200146e-02,
         5.04499972e-02,  5.87419987e-01,  1.61250010e-01,
        -1.27790004e-01, -5.09299934e-02,  5.82489967e-02,
        -1.54311001e-01, -2.74599999e-01, -4.05699909e-02,
         4.65460002e-01, -2.69311011e-01, -6.96425974e-0

## Conclusion

- Total of 3131 adjective antonym pairs plus 1 or more custom pairs
- Each pair must have GloVe embeddings (i.e. embedding not [0,0,0,0,0, ..., 0])

# 2. Contribution of a Word to Microframes

For calculating cosine similarity, see [documentaion](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) from sklearn

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# example:
x = np.array([[1,2,2]])
y = np.array([[3,4,1]])
cosine_similarity(x,y).item()

0.8498365855987975

In [23]:
np.expand_dims(glove['man'], axis=0).shape
cosine_similarity(np.expand_dims(glove['man'], axis=0), np.expand_dims(glove['woman'], axis=0)).item()

0.740174412727356

### Wrapper for cosine similarity
Given two words, find similarity:

In [24]:
def my_cos_similarity(func):
    def wrapper(word, axis_vector):
        return func(np.expand_dims(glove[word], axis=0), axis_vector)
    return wrapper

@my_cos_similarity
def cos_similarity(x, y):
    return cosine_similarity(x, y).item()

cos_similarity('woman',semantic_axis_vectors['(man, woman)'])

0.4572717547416687

The **absolute value** of the similarity between a word vector and
a microframe vector captures the relevance of the word to the
microframe, while the **sign** of the similarity captures a bias toward
one of the poles in the microframe.

# 3. Framing Bias and Intensity

## Bias and word frequency

In [25]:
import nltk
from multiprocessing import Pool
from tqdm.notebook import tqdm

In [26]:
# f
# word is a single word string
# doc_tokens is a list of all document tokens

def word_freq(word, doc_tokens):
    return doc_tokens.count(word)

### Regular function

In [27]:
# B
def bias(doc, frame=None):
    doc_tokens = nltk.word_tokenize(doc)
    
    numerator, denominator = 0, 0
    for word in doc_tokens:
        numerator += word_freq(word, doc_tokens) * cos_similarity(word, semantic_axis_vectors[frame])
        denominator += word_freq(word, doc_tokens)
        
    return numerator / denominator

In [28]:
%%timeit
sample = data_cleaned['RS_2020_nosleep'].iloc[0:1].selftext.item()
bias(sample, frame='(man, woman)')

97.7 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Same function, but with performance enhancement

In [29]:
%%timeit
sum1, sum2 = 0, 0
for i in range(1_000_000):
    sum1 += 1
    sum2 += 2
sum1 / sum2

67.1 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit
sum1, sum2 = sum([1 for i in range(1_000_000)]), sum([2 for i in range(1_000_000)])
sum1 / sum2

54.6 ms ± 831 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
# B
def bias(doc, frame=None):
    doc_tokens = nltk.word_tokenize(doc)
    numerator, denominator = sum([word_freq(word, doc_tokens) * cos_similarity(word, semantic_axis_vectors[frame]) for word in doc_tokens]), sum([word_freq(word, doc_tokens) for word in doc_tokens])
        
    return numerator / denominator

In [32]:
%%timeit
sample = data_cleaned['RS_2020_nosleep'].iloc[0:1].selftext.item()
bias(sample, frame='(man, woman)')

99.3 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# $B^T$ (failed for now)

In [33]:
import pandas as pd
concat_df = pd.concat([data_cleaned['RS_2020_self'], data_cleaned['RS_2020_nosleep']])

In [34]:
np.savetxt("Download/concat_texts.txt", concat_df['selftext'].astype('string').values, fmt='%s')

In [35]:
with open('Download/concat_texts.txt', 'r') as file:
    concat_texts = file.read()

In [35]:
# bias(concat_texts, frame = 'man - woman')

## Intensity (failed for now)

### Create the entire corpus *T*

In [45]:
# I
def intensity(doc, frame=None, corpus = concat_texts):
    doc_tokens = nltk.word_tokenize(doc)
    
    print('Calculating B^T')
    B_T = bias(corpus, frame = frame)
    print('Done')
    numerator, denominator = sum([word_freq(word, doc_tokens) * (cos_similarity(word, semantic_axis_vectors[frame]) - B_T)**2 for word in doc_tokens]), sum([word_freq(word, doc_tokens) for word in doc_tokens])
        
    return numerator / denominator

In [36]:
sample = data_cleaned['RS_2020_nosleep'].iloc[0:1].selftext.item()
# intensity(sample, frame='man - woman', corpus = concat_texts)

# 4. Statistical Significance of Microframes

In [36]:
import pandas as pd
import numpy as np

def df_mframes(df, microframes=microframes, topn=None):
    # if not specify topn
    if not topn:
        iterables = [df['id'].to_list(), list(microframes)]
        index = pd.MultiIndex.from_product(iterables, names=['id', 'microframe'])
        df_new = pd.DataFrame(np.repeat(df[['title', 'selftext', 'score']].values, 
                                    len(microframes), 
                                    axis = 0),
                          index=index,
                          columns=['title', 'selftext', 'score']).copy()
        # map semantic axis vector to index
        # how it works: index -> get 2nd level index -> to series -> convert tuple to string -> map values
        a = df_new.index.get_level_values(1).to_series().apply(lambda row: f"({row[0]}, {row[1]})").map(semantic_axis_vectors)
        df_new['Semantic Axis Vector'] = a.to_list()
        return df_new
    else:
        iterables = [df['id'].head(topn).to_list(), list(microframes)]
        index = pd.MultiIndex.from_product(iterables, names=['id', 'microframe'])
        df_new = pd.DataFrame(np.repeat(df[['title', 'selftext', 'score']].head(topn).values, 
                                    len(microframes), 
                                    axis = 0),
                          index=index,
                          columns=['title', 'selftext', 'score']).copy()
        # map semantic axis vector to index
        # how it works: index -> get 2nd level index -> to series -> convert tuple to string -> map values
        a = df_new.index.get_level_values(1).to_series().apply(lambda row: f"({row[0]}, {row[1]})").map(semantic_axis_vectors)
        df_new['Semantic Axis Vector'] = a.to_list()
        return df_new

In [37]:
sample_doc = data_cleaned['RS_2020_nosleep'].iloc[0:1].selftext.item()

In [38]:
sample_doc

"This is the only rule of our household. If you feel a presence standing over you while you sleep, do **NOT** open your eyes. Ignore it and try to fall asleep. This may sound a bit like the show Bird Box from Netflix but this is real, this is serious. You don't want to know what happens if you open your eyes. Let me start from the beginning, from where it all began.\n\n------------------------\n\nMy family had just moved to a new house in a new city in an eerily quiet neighborhood. We barely ever saw the neighbors and there were almost no animals around. It threw me off a little bit but i quickly got accustomed to it. My school was pretty close to my house so it was not difficult to make friends that I could regularly bring home. So, a few months in the new house, when i was sleeping one night, i felt a figure standing over me. Now, I have always been a bit of a scaredy cat so I slammed my eyes shut and waited. A few moments later, I felt it go away. This happened to me every night and

In [39]:
with open('Download/concat_texts.txt', 'r') as file:
    concat_texts = file.read()
    T = nltk.word_tokenize(concat_texts)

In [40]:
len(nltk.word_tokenize(sample_doc))

575

In [59]:
from sklearn.utils import resample
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

N = 140
t = len(nltk.word_tokenize(sample_doc))
df = pd.DataFrame(columns=microframes)
for i in tqdm(range(N)):
    s = resample(T, replace=True, n_samples=t)
    s = ' '.join(s)
    biases = []
    for ant1, ant2 in tqdm(microframes):
        biases.append(bias(s, frame=f"({ant1}, {ant2})"))
        
    df.loc[i] = biases
    
df

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/3133 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [155]:
resample(T, replace=True, n_samples=len(nltk.word_tokenize(sample_doc)))

['It',
 'schoolmates',
 'I',
 'goodbye',
 'ever',
 'took',
 'of',
 'were',
 '.',
 'see',
 'the',
 'they',
 'a',
 'Harrison',
 'seen',
 'dark',
 'said',
 'up',
 'love',
 'going',
 'the',
 'wearing',
 'I',
 'homeless',
 'nevertheless',
 'the',
 'and',
 'It',
 'hot',
 'of',
 'deep',
 'We',
 '.',
 'they',
 'I',
 'ringing',
 'up',
 'crossed',
 'Burt',
 'that',
 'finagling',
 'right',
 'dragon',
 'in',
 "''",
 '.',
 'would',
 'feel',
 'down',
 'Ace',
 'know',
 'my',
 'for',
 'the',
 'go',
 'turn',
 'us',
 'my',
 'noise',
 'lie',
 'there',
 'rustling',
 ',',
 'to',
 'mind',
 'said',
 'me',
 ',',
 'our',
 'There',
 'I',
 'be',
 'again',
 'reached',
 'thirteen',
 'me',
 'minutes',
 'went',
 'at',
 'future',
 'nighter',
 'doctor',
 'to',
 'heard',
 'choice',
 'overcame',
 'me',
 'my',
 ',',
 'my',
 'watched',
 '.',
 'we',
 'were',
 'that',
 '``',
 'a',
 '.',
 'want',
 '.',
 'is',
 'pissing',
 'of',
 'I',
 'Our',
 '*',
 'Oh',
 'in',
 'he',
 'ran',
 'began',
 'to',
 'hugging',
 'acre',
 "n't",
 't

In [52]:
df = pd.DataFrame(columns=microframes)
df

,unpopular,lively,apocrine,fixed,inexpressible,airworthy,unmalted,communicative,uninhibited,punitive,...,gutsy,ventral,colored,hypertrophied,exclusive,dominant,low-tension,eccrine,ill-natured,pro
,popular,dull,eccrine,unfixed,expressible,unairworthy,malted,uncommunicative,inhibited,rehabilitative,...,gutless,dorsal,uncolored,atrophied,inclusive,subordinate,high-tension,apocrine,good-natured,anti


In [53]:
microframes

{('unpopular', 'popular'),
 ('lively', 'dull'),
 ('apocrine', 'eccrine'),
 ('fixed', 'unfixed'),
 ('inexpressible', 'expressible'),
 ('airworthy', 'unairworthy'),
 ('unmalted', 'malted'),
 ('communicative', 'uncommunicative'),
 ('uninhibited', 'inhibited'),
 ('punitive', 'rehabilitative'),
 ('paid', 'unpaid'),
 ('leaky', 'tight'),
 ('prudent', 'imprudent'),
 ('untruthful', 'truthful'),
 ('precise', 'imprecise'),
 ('wieldy', 'unwieldy'),
 ('unlisted', 'listed'),
 ('aperiodic', 'periodic'),
 ('nearsighted', 'farsighted'),
 ('crosswise', 'lengthwise'),
 ('unoiled', 'oiled'),
 ('thin', 'thick'),
 ('modulated', 'unmodulated'),
 ('irremediable', 'remediable'),
 ('divisible', 'indivisible'),
 ('uncrossed', 'crossed'),
 ('awake', 'asleep'),
 ('artless', 'artful'),
 ('sorrowful', 'joyful'),
 ('unclean', 'clean'),
 ('synchronous', 'asynchronous'),
 ('thermal', 'nonthermal'),
 ('agitated', 'unagitated'),
 ('anestrous', 'estrous'),
 ('efferent', 'afferent'),
 ('lengthwise', 'crosswise'),
 ('unfree

In [48]:
df = pd.DataFrame(columns=('A', 'B'))
for i in range(3):
    df.loc[i] = [i, i + 1]

print(df)

   A  B
0  0  1
1  1  2
2  2  3
